In [99]:
import random
import os
import numpy as np
import copy
from time import time

In [62]:
# dont seed since we want new stuff each runthrough

In [63]:
clans = {
    'Banu Haqum',
    'Brujah',
    'Gangrel',
    'Hecata',
    'Lasombra',
    'Malkavian',
    'Ministry',
    'Nosferatu',
    'Ravnos',
    'Toreador',
    'Tremere',
    'Tzimisce',
    'Ventrue'
}

In [64]:
competency = {
    'beginner': []
}

In [65]:
def roll(n:int=1):
    return int(sum([(random.random().__round__(0)) for _ in range(n)]))

In [89]:
class Character:
    def __init__(self, name, atk, df, spd, hp, hunger=0):
        self.name = name
        self.atk = atk
        self.df = df
        self.spd = self.roll(spd)
        self.total_hp = hp
        self.hunger = hunger
        self.dmg = 0
        self.agg_dmg = 0
        self.is_alive = True

    @staticmethod
    def roll(n:int=1):
        return int(sum([(random.random().__round__(0)) for _ in range(n)]))

    def attack(self, other_character):
        if not self.is_alive or not other_character.is_alive : return
        dmg = self.roll(self.atk) - other_character.roll(other_character.df)
        if dmg > 0:
            other_character.take_damage(dmg)
        else:
            self.take_damage(-dmg)

    def take_damage(self, n, aggravated=False):
        if aggravated:
            self.agg_dmg += n
        else:
            self.dmg += n
            if self.dmg > self.total_hp:
                self.agg_dmg += self.dmg - self.total_hp
                self.dmg = self.total_hp

        self.is_alive = self.total_hp > self.agg_dmg
        return self.is_alive

    def reset(self):
        self.dmg = 0
        self.agg_dmg = 0

In [67]:
def getspd(character:Character):
    return -character.spd

In [68]:
vamp = Character('vamp', 6, 4, 4, 7, 0)
ww = Character('ww', 9, 8, 6, 10, 0)

fighters = [vamp, ww]
fighters = sorted(fighters, key=getspd)

print([(f.name, f.spd) for f in fighters])

[('vamp', 2), ('ww', 2)]


In [69]:
def combat(party1:list, party2:list, wins=0, counter=None):
    if counter is None:
        counter = dict([(c.name, 0) for c in party1])
    party_dict = {
        1:party1,
        -1:party2
    }

    all_chars = sorted(party1 + party2, key=getspd)
    all_chars = [(k, 1) if k in party1 else (k, -1) for k in all_chars]
    while len(party1) > 0 and len(party2) > 0:
        for char, key in all_chars:
            if len(party1) == 0 or len(party2) == 0: break
            assert char.is_alive, f'{char.name} is dead with hitpoints {char.total_hp - char.agg_dmg} but is still attacking'
            attack_party = party_dict[key]
            defend_party = party_dict[-key]
            defender:Character = random.choice(defend_party)
            char.attack(defender)
            if not char.is_alive:
                attack_party.remove(char)
                all_chars.remove((char, key))
            if not defender.is_alive:
                defend_party.remove(defender)
                all_chars.remove((defender, -key))


    for c in party1:
        counter[c.name] += 1
    if len(party1) > len(party2):
        # print('party 1 wins!')
        # print([(f.name, f.total_hp - f.agg_dmg) for f in party1])
        return wins + 1, counter
    else:
        # print('party 2 wins!')
        # print([(f.name, f.total_hp - f.agg_dmg) for f in party2])
        return wins, counter

In [104]:
t = time()
# party1 = [Character('DK', atk=4, df=4, spd=5, hp=7),
#           Character('chris', atk=9, df=6, spd=4, hp=8),
#           Character('rabbit', atk=9, df=5, spd=4, hp=10)]
# party2 = [Character('ww', 10, 8, 6, 10)]
counter = None

num_iters = 10000
wins = 0
for _ in range(num_iters):
    party1 = [Character('DK', atk=4, df=4, spd=5, hp=7),
              Character('chris', atk=9, df=6, spd=4, hp=8),
              Character('rabbit', atk=9, df=5, spd=4, hp=10)]
    party2 = [Character('ww', 10, 8, 6, 10)]
    wins, counter = combat(party1, party2, wins, counter)
print(f'{int(100 * wins / num_iters)}%')
for k,v in counter.items():
    print(f'{k}:{int(100 * v / num_iters)}%')
print(f'seconds elapsed: {(time() - t).__round__(1)}s')

67%
DK:7%
chris:58%
rabbit:61%
seconds elapsed: 3.1s


In [110]:
def pick_enemy_attributes(party:list, enemy:Character, num_iters:int=30, inner_num_iters:int=1000) -> (Character, float):

    atk = enemy.atk,
    df = enemy.df
    hp = enemy.total_hp
    spd = enemy.spd

    vars = [atk, df, hp]

    wins = 0
    for i in range(num_iters):
        counter = None
        wins = 0
        for _ in range(inner_num_iters):
            party1 = copy.deepcopy(party)
            party2 = [Character(enemy.name, atk, df, spd, hp, hunger=0)]
            wins, counter = combat(party1, party2, wins, counter)
        wins /= num_iters
        change = 0
        if wins < .75: change = -1
        elif wins > .85: change = 1
        else: return Character(enemy.name, atk, df, spd, hp, hunger=0), wins
        v = random.choice(vars)
        v += change
    return Character(enemy.name, atk, df, spd, hp, hunger=0), wins

In [111]:
party = [Character('DK', atk=4, df=4, spd=5, hp=7),
          Character('chris', atk=9, df=6, spd=4, hp=8),
          Character('rabbit', atk=9, df=5, spd=4, hp=10)]
enemy = Character('ww', 10, 8, 6, 10)
num_iters = 30
inner_num_iters = 1000
final_enemy, wins = pick_enemy_attributes(party, enemy, num_iters, inner_num_iters)
print(f'{(100 * wins).__round__(1)}%')
print(final_enemy.name, final_enemy.atk, final_enemy.df, final_enemy.spd, final_enemy.total_hp)

TypeError: 'tuple' object cannot be interpreted as an integer

In [73]:
# party1 = np.array([
#     Character('DK', atk=4, df=4, spd=5, hp=7),
#     Character('chris', atk=9, df=6, spd=4, hp=8),
#     Character('rabbit', atk=9, df=5, spd=4, hp=10)
# ])
# party2 = np.array([Character('ww', 10, 8, 6, 12)])
# all_chars = sorted(np.r_[party1,party2], key=getspd)
# counts = np.array([c.atk for c in party2])
# counts = counts / counts.sum()

In [74]:
# butt = {'butt', 'aids', 'poop'}
#
# random.sample(butt, k=1)

In [75]:
# party1 = {'a','b','c'}
# party2 = {'jewfag'}
# p = party1
# p.remove('a')
# party1

In [76]:
# party1 = np.array([
#     Character('DK', atk=4, df=4, spd=5, hp=7),
#     Character('chris', atk=9, df=6, spd=4, hp=8),
#     Character('rabbit', atk=9, df=5, spd=4, hp=10)
# ])
# party2 = np.array([Character('ww', 10, 8, 6, 12)])
# counter = None
#
# all_chars = dict.fromkeys(party1, 1)
# update_dict = dict.fromkeys(party2, 2)